# Mapping Longitude and Latitude to GeoJSON polygon

## Reference:
https://stackoverflow.com/questions/20776205/point-in-polygon-with-geojson-in-python

In [1]:
import csv
import json
from shapely.geometry import shape, polygon, Point
from shapely.strtree import STRtree
mappath = 'nyu-2451-36743-geojson.json'
readpath = '2013_Green_head_50K.csv'
writepath = '2013_Green_map.csv'

In [2]:
with open(mappath) as fj:
        js = json.load(fj)

# generate geo rtree
polys = []    
for feature in js['features']:
    polygon = shape(feature['geometry'])
    # poly=STRtree(polygon.geoms)
    poly=STRtree(polygon)
    polys += [poly]


In [3]:
def search_location(point):
    ipos=0
    fflag=False

    for poly in polys:
        po = poly.query(point)
        if po != []:
            fflag=True
            break
        ipos+=1

    if fflag:
        return js['features'][ipos]['properties']['locationid']
    else:
        return 0 # empty location

In [4]:
def cal_zone2(mappath,readpath,writepath,windows=True):
    axis = ('PU_Lon','PU_Lat','DO_Lon','DO_Lat')
    
    with open(readpath,'r') as rf, open(writepath,'w') as wf:
        reader=csv.reader(rf)
        writer=csv.writer(wf, lineterminator='\n')
    
        # read header
        head=next(reader)
        nf=len(head)   # number of fileds
   
        writer.writerow(['row_id','PULocationID','DOLocationID'])
        
        for i in range(nf):
            if head[i] == axis[0]:
                pu_lon_ = i
            elif head[i] == axis[1]:
                pu_lat_ = i
            elif head[i] == axis[2]:
                do_lon_ = i
            elif head[i] == axis[3]:
                do_lat_ = i

        row_cnt=0
        row_id = 0
        
        # process data
        for row in reader:
            row_id += 1
            
            point1 = Point(float(row[pu_lon_]), float(row[pu_lat_]))
            point2 = Point(float(row[do_lon_]), float(row[do_lat_]))
            
            zone1 = search_location(point1)
            zone2 = search_location(point2)
            
            new_row = [row_id,zone1,zone2]
            writer.writerow(new_row)
            
            row_cnt+=1
            
        print('rows:',row_cnt)
   

In [5]:
%time cal_zone2(mappath,readpath,writepath)

rows: 50000
CPU times: user 26 s, sys: 105 ms, total: 26.1 s
Wall time: 26.2 s


### Original code to map (Longitude,Latitude) set to NYC Taxi Zones

In [6]:
# load GeoJSON file containing sectors
with open('nyu-2451-36743-geojson.json') as f:
    js = json.load(f)

In [7]:
len(js['features'])

263

In [ ]:
# 讀取全部 263 區域
# js['features']

In [ ]:
js['features'][21]

In [8]:
# construct point based on lon/lat returned by geocoder
# 隨便給一個點的經緯度
#point = Point(-73.9919569, 40.721567)
point = Point(-73.913970947265625,40.773616790771484)

In [9]:
type(point)

shapely.geometry.point.Point

In [10]:
# check each polygon to see if it contains the point
# 看看剛剛給的點是不是在某一個區域中

for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        zone = feature['properties']['locationid']

In [11]:
zone

223

In [12]:
search_location(point)

223